# Lab Assignment - 7

Train the AlexNet for the MNIST dataset (digits 0–9) and the CIFAR-10 dataset using PyTorch. Plot the loss and accuracy. Tabulate and compare the performance of AlexNet on the MNIST dataset with the performance you got for the previous lab exercises.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# -------------------------------
# AlexNet Implementation
# -------------------------------
class AlexNet(nn.Module):
    def __init__(self, num_classes=10, in_channels=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*3*3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [2]:
# CIFAR-10 (RGB)
transform_cifar = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

# MNIST (grayscale → 1 channel, upscaled to 32x32)
transform_mnist = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

# Dataloaders
trainloader_cifar = torch.utils.data.DataLoader(cifar_train, batch_size=64, shuffle=True)
testloader_cifar = torch.utils.data.DataLoader(cifar_test, batch_size=64, shuffle=False)

trainloader_mnist = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
testloader_mnist = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:05<00:00, 29.5MB/s]
100%|██████████| 9.91M/9.91M [00:00<00:00, 34.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.56MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.0MB/s]


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, trainloader, testloader, name, epochs=5, lr=0.001):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses, test_losses, train_acc, test_acc = [], [], [], []

    for epoch in range(epochs):
        # Training
        model.train()
        correct, total, loss_sum = 0, 0, 0
        for x, y in trainloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)
        train_losses.append(loss_sum/len(trainloader))
        train_acc.append(correct/total)

        # Testing
        model.eval()
        correct, total, loss_sum = 0, 0, 0
        with torch.no_grad():
            for x, y in testloader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)
                loss_sum += loss.item()
                _, pred = out.max(1)
                correct += pred.eq(y).sum().item()
                total += y.size(0)
        test_losses.append(loss_sum/len(testloader))
        test_acc.append(correct/total)

        print(f"Epoch {epoch+1}/{epochs} | Train Acc: {train_acc[-1]*100:.2f}% | Test Acc: {test_acc[-1]*100:.2f}%")

    # Plot Loss/Accuracy
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(train_losses, label="train_loss")
    plt.plot(test_losses, label="test_loss")
    plt.title(f"{name} Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(train_acc, label="train_acc")
    plt.plot(test_acc, label="test_acc")
    plt.title(f"{name} Accuracy")
    plt.legend()
    plt.show()

    return max(test_acc)


In [4]:
# MNIST (grayscale = 1 channel)
alexnet_mnist = AlexNet(num_classes=10, in_channels=1)
acc_mnist = train_model(alexnet_mnist, trainloader_mnist, testloader_mnist, "AlexNet on MNIST", epochs=5)

# CIFAR-10 (RGB = 3 channels)
alexnet_cifar = AlexNet(num_classes=10, in_channels=3)
acc_cifar = train_model(alexnet_cifar, trainloader_cifar, testloader_cifar, "AlexNet on CIFAR-10", epochs=10)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x4096 and 2304x4096)